In [ ]:
# This python notebook annotate the bioconda recipes using the ids from bio.tools in the internal local path to the 
# in bioconda recipes. Two different input should be provided: 
# @param bioconda_recipes_path the local path to the bioconda recipes, this can be download using the git clone command
# @param mapping_file is the local path to the mapping file generated with the notebook biotools-biocontainers-venn.ipynb


import glob2
import os
import sys

from pandas import DataFrame, read_csv

from ruamel.yaml import YAML

from yaml.constructor import ConstructorError
from yaml.scanner import ScannerError

In [ ]:
def init_yaml_parser():
    yaml = YAML(typ='jinja2')
    yaml.allow_duplicate_keys = True
    yaml.explicit_start = True
    yaml.
    return yaml

In [ ]:
def get_bioconda_yalm(bioconda_path, yaml_parser):
    yaml_files = dict()
    matches = glob2.glob(bioconda_path + '/**/*.yaml', with_matches=True)
    if len(matches) > 0:
        for file_name in matches:
            if (os.path.isfile(file_name[0])):
                # print(file_name[0])
                path_list = file_name[0].replace(bioconda_path, "").split(os.sep)
                software_name = path_list[1]
                try:
                    with open(file_name[0]) as fp:
                        yalm_file = yaml_parser.load(fp)
                    yaml_files[software_name] = (file_name[0], yalm_file)
                    #print(yaml_files[software_name])
                except (ScannerError, ConstructorError, TypeError, AttributeError) as error:
                    print("ERROR | " + software_name + "\t" + file_name[0])
    return yaml_files         

In [ ]:
def get_annotation_mapping(mapping_file):
    mapping = read_csv(mapping_file)
    return mapping    

In [ ]:
def annotate_yaml(yaml_file, biotool, yaml_parser):
    path_file = yaml_file[0]
    print(yaml_file)
    yaml_file_content = yaml_file[1]
    if yaml_file_content.get('extra') is None:
        yaml_file_content['extra'] = {}
    else:
        print("EXTRA has been found: " + biotool)
    yaml_file_content['extra']['biotools'] = [biotool]

    with open(path_file, 'w') as fp:
        yaml_parser.dump(yaml_file_content, fp)

    

def annotate_yamls(yaml_dictonary, mappings, yaml):
    for (idx, row) in mappings.iterrows():
        if yaml_dictonary.get(row.bioconda) is not None:
            print(yaml_dictonary.get(row.bioconda))
            annotate_yaml(yaml_dictonary[row.bioconda], row.biotools, yaml)

In [ ]:
bioconda_recipes_path = "/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes"
mapping_file = "/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-biotools/mapping_matchonly.csv"
yaml_parser = init_yaml_parser()
mapping = get_annotation_mapping(mapping_file)
yaml_files = get_bioconda_yalm(bioconda_recipes_path, yaml_parser)
annotate_yamls(yaml_files, mapping, yaml_parser)